# Aula 3 - Limpeza e transformação de dados

Na aula de hoje, vamos explorar os seguintes tópicos em Python:

- 1) Limpeza de Dados (dropna, fillna, isnull, notnull, replace, duplicated, drop_duplicated)
- 2) Transformação de Dados (cut, qcut, get_dummies)

________

### Objetivos

Apresentar como identificamos valores nulos, discutir melhores formas de tratá-los além de trabalhar com variáveis dummies.

________

### Habilidades a serem desenvolvidas nessa aula

Ao final da aula o aluno deve:

- Saber como identificar valores nulos e como tratá-los.
- Identificar quando utilizar variáveis dummies e como criá-las.
____
____
____

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("data/titanic.csv")

## Limpeza de Dados: Dados Faltantes

### `.isna() e isnull()`
retorna um df booleano indicando se existe um nulo naquela posição. `pd.isnull()` é um alias para `pd.isna()` dentro do python como podemos ver na <a href="https://github.com/pandas-dev/pandas/blob/0409521665bd436a10aea7e06336066bf07ff057/pandas/core/dtypes/missing.py#L109">documentação</a>. O pandas dataframe é baseado nos df do R, onde null e na tem sentidos distintos.

In [ ]:
pd.isnull

Tanto o `.isna()` quanto o `.isnull()` retornam um df booleano:

In [ ]:
df.isna()

Para termos a quantidade de nans em cada coluna podemos somar os valores true:

### `.notnull()`

retorna df com True ou False, com True para elementos não nulos

### `.dropna()`

elimina nulos em todo o df ou em colunas especificadas quando utilizado o argumento `subset`. É um dos métodos que aceita o parâmetro `inplace`.


In [ ]:
df.dropna(subset=['Age','Embarked'])

In [ ]:
df.dropna(subset=['Age','Embarked']).isna().sum()

### `.fillna()`

preeche os valores nulos com valores especificados.
Parâmetros úteis:

   * method {‘bfill’,‘ffill’, None}, default None <br>
       Preenche os nas propagando o último valor válido para frente (ffill) ou utiliza próxima observação válida para preencher os nulos (bfill).
   * axis {0 para ‘index’, 1 para ‘colunas’} <br>
       Eixo no qual se quer preencher os nulos
   * inplace {booleano}, default False


In [ ]:
fill = pd.DataFrame({'original': [np.nan,1,2,np.nan,4,5,np.nan,np.nan, np.nan, 9, 10, np.nan]})
fill

In [ ]:
fill['cte'] = fill.original.fillna(999)
# fill['media'] = fill.original.fillna(fill.original.mean())
# fill['ffill'] = fill.original.fillna(method='ffill')
# fill['ffill_limit'] = fill.original.fillna(method='ffill', limit=2)
# fill['bfill'] = fill.original.fillna(method='bfill')

fill

### Voltando ao dataset do titanic...
Quais soluções poderíamos utilizar para preencher os nulos em Age? <br>
E as cabines nulas? <br>
E os portões de embarque nulos? <br>
Qual os possíveis problemas que podemos encontrar utilizando essas abordagens? <br>


_________
_________
**Exercício:** Ordene as colunas Pclass e Fare, nessa ordem e ascendente, e preencha os nulos utilizando o método foward fill.

In [3]:
df.sort_values(["Pclass", "Fare"], ascending = True).fillna(method = "ffill")

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
263,264,0,1,"Harrison, Mr. William",male,40.0,0,0,112059,0.00,B94,S
633,634,0,1,"Parr, Mr. William Henry Marsh",male,40.0,0,0,112052,0.00,B94,S
806,807,0,1,"Andrews, Mr. Thomas Jr",male,39.0,0,0,112050,0.00,A36,S
815,816,0,1,"Fry, Mr. Richard",male,39.0,0,0,112058,0.00,B102,S
822,823,0,1,"Reuchlin, Jonkheer. John George",male,38.0,0,0,19972,0.00,B102,S
...,...,...,...,...,...,...,...,...,...,...,...,...
201,202,0,3,"Sage, Mr. Frederick",male,32.0,8,2,CA. 2343,69.55,F E69,S
324,325,0,3,"Sage, Mr. George John Jr",male,32.0,8,2,CA. 2343,69.55,F E69,S
792,793,0,3,"Sage, Miss. Stella Anna",female,32.0,8,2,CA. 2343,69.55,F E69,S
846,847,0,3,"Sage, Mr. Douglas Bullen",male,32.0,8,2,CA. 2343,69.55,F E69,S


_________
_________

### `.interpolate()`
Faz a interpolação dos missing de acordo com o método escolhido.
[Documentação](https://pandas.pydata.org/pandas-docs/version/0.23/generated/pandas.Series.interpolate.html)

In [ ]:
s = pd.DataFrame([0, 1, np.nan, 3, 4, np.nan, 400], columns=['original'])
s['linear'] = s.original.interpolate()
s['segundo_grau'] = s.original.interpolate(method="polynomial", order=2)
s

### `.replace()`

substitui elementos dentro do df. É um dos métodos que aceita o parâmetro `inplace`.


In [ ]:
# para apenas uma troca


In [ ]:
# para mais de uma troca
df.replace({'Sex':{'male':'H', 'female':'M'}})

In [ ]:
df.isna().sum()

___________________
___________________

**Exercício**: Troque os valores abreviados do portão de embarque para seus respectivos nomes 
(C = Cherbourg; Q = Queenstown; S = Southampton) e preencha os dados faltantes com o portão mais frequente

In [101]:
# Resposta
df.Embarked.replace({"C":"Cherbourg", "Q":"Queenstown", "S":"Southampton"}, inplace=True)
df["Embarked"].fillna(df["Embarked"].mode()[0], inplace=True)

In [102]:
df["Embarked"].isna().sum()

0

___________________
___________________


## Limpeza de Dados: Dados duplicados
### `.duplicated()` e `.drop_duplicated()`

O `.duplicated()` retorna uma series indicando se determinada linha possui duplicados ou não. Já o `.drop_duplicated()`, elimina as linhas duplicadas. <br>
Parâmetros do `.drop_duplicated()`:

   * `subset` -  seleciona colunas para serem utilizadas na comparação de linhas duplicadas
   * `keep` - {‘first’, ‘last’, False} (default ‘first’) Determina qual duplicado manter
   * `inplace` - (default False) Se False retorna uma cópia do df com as alterações, se True faz as modificações no próprio df



In [ ]:
# vamos criar um df que possua linhas duplicadas
df_dup = df.copy()

df_dup.loc[891, :] = df_dup.loc[0, :]
df_dup.loc[892, :] = df_dup.loc[1, :]
df_dup.loc[893, :] = df_dup.loc[1, :]

print(df.shape, df_dup.shape)

In [ ]:
df_dup.duplicated().sum()

In [ ]:
print(df_dup.shape)
print(df_dup.drop_duplicates().shape)

_______________________
_______________________
**Exercícios:** Elimine os duplicados considerando as colunas 'brand' e 'style' mantendo o último valor.

In [42]:
df_exerc = pd.DataFrame({
    'brand': ['Yum Yum', 'Yum Yum', 'Yum Yum', 'Indomie', 'Indomie', 'Indomie'],
    'style': ['cup', 'cup', 'cup', 'cup', 'pack', 'pack'],
    'rating': [4, 4, np.nan, 3.5, 15, 5]
})
df_exerc

,brand,style,rating
0,Yum Yum,cup,4.0
1,Yum Yum,cup,4.0
2,Yum Yum,cup,NaN
3,Indomie,cup,3.5
4,Indomie,pack,15.0
5,Indomie,pack,5.0


In [45]:
df_exerc.drop_duplicates(["brand", "style"], keep="last", inplace=True)
df_exerc

,brand,style,rating
2,Yum Yum,cup,NaN
3,Indomie,cup,3.5
5,Indomie,pack,5.0


__________
__________


## Transformação de dados

### `.get_dummies()`

#### variáveis categóricas
Variáveis categóricas são aquelas que representam grupos ou classes dentro dos nossos dados. Elas podem ser de dois tipos:
* ordinais: possuem uma ordem que tem um sentido. Por exemplo, em rendimentos poderíamos ter: classe alta > classe média > classe baixa  
* nominais: não possuem uma ordem válida. Por exemplo: sexo e CEP.

<img src="variaveis_categoricas.jpeg" style="width: 500px">

Dummies são quaisquer variáveis cujos valores são 1 ou 0 para cada observação. O método `pd.get_dummies()` converte as variáveis categóricas em numéricas separando cada categoria em uma coluna única.
<br>
<br>
<a href="https://pandas.pydata.org/docs/reference/api/pandas.get_dummies.html">Parâmetros:</a> <br>
`pandas.get_dummies(data, prefix=None, prefix_sep='_', dummy_na=False, columns=None, sparse=False, drop_first=False, dtype=None)`

## Material de Aprofundamento

### `.cut()`
O método `.cut()` ordena os dados, separa em bins e computa qual grupo cada linha do df pertence. O `.cut()` escolherá os bins para serem espaçados uniformemente de acordo com os próprios valores e não com a frequência desses valores.  <br>
Ele é muito utilizado para transformar variáveis contínuas em categóricas. Por exemplo, podemos converter o valor númerico da idade em grupos de criança, jovem, adulto e idoso.
<br><br>
<a href='https://pandas.pydata.org/docs/reference/api/pandas.cut.html'>Parâmetros:</a> <br>
`pandas.cut(x, bins, right=True, labels=None, retbins=False, precision=3, include_lowest=False, duplicates='raise', ordered=True)`

Ao informar quantidade de grupos o pd.cut() escolhe os bins com o mesmo tamanho de janela :


In [ ]:
df['cut_bins'] = pd.cut(df.Age, 4)
df.head()

In [ ]:
df.cut_bins.value_counts()

Podemos passar o nome dos grupos e transformar a variável numérica diretamente em categórica


In [ ]:
df['cut_classes'] = pd.cut(df.Age, 4, labels=["jovens", "adultos", "meia-idade", "idosos"])
df.head()

In [ ]:
df.cut_bins.unique()

In [ ]:
df.cut_classes.value_counts()

Também podemos passar uma lista com os valores de início e fim dos bins:

In [ ]:
pd.cut(df.Age, [0,20,60,80]).unique()

In [ ]:
df.Age.describe()

<a href='https://towardsdatascience.com/discretisation-using-decision-trees-21910483fa4b'>Discretização utilizando decision trees</a>

### `.qcut()`
O `pd.qcut()` é utilizado quando queremos discretizar nossos dados em quantis. Ao informar quantidade de grupos o `pd.qcut()` escolhe os bins tal que tenhamos a mesma quantidade de valores em cada grupo.

#### `.cut()` x `.qcut()`
   * O comando `.cut()` cria **caixas equidistantes**, mas a **frequência** das amostras é **desigual** em cada caixa
   * O comando `.qcut()` cria **caixas de tamanhos desiguais**, mas a **frequência** das amostras é **igual** em cada caixa.

<br>

[Parâmetros](https://pandas.pydata.org/docs/reference/api/pandas.qcut.html)
<br>
`pandas.qcut(x, q, labels=None, retbins=False, precision=3, duplicates='raise')`

In [ ]:
pd.cut(df.Age, 4).value_counts()

In [ ]:
pd.qcut(df.Age, 4).value_counts()

In [ ]:
pd.qcut(df.Age, 4).value_counts()/df.Age.notnull().sum()

## Exercícios

1. Converta os países do dataframe abaixo para variáveis dummies eliminando a primeira coluna e utilizando 'Country' como prefixo no nome das colunas.

In [46]:
ids = [11, 22, 33, 44, 55, 66, 77]
countries = ['Brasil', 'Argentina', 'Peru', 'Bolívia', 'Uruguai']
sales = [42, 53, 37, 45, 20]

df = pd.DataFrame(list(zip(ids, countries, sales)),
                  columns=['Ids', 'Countries', 'Sales'])
df.head()

,Ids,Countries,Sales
0,11,Brasil,42
1,22,Argentina,53
2,33,Peru,37
3,44,Bolívia,45
4,55,Uruguai,20


In [48]:
df = pd.get_dummies(df, prefix="Country", prefix_sep='_', columns=["Countries"])
df

,Ids,Sales,Country_Argentina,Country_Bolívia,Country_Brasil,Country_Peru,Country_Uruguai
0,11,42,0,0,1,0,0
1,22,53,1,0,0,0,0
2,33,37,0,0,0,1,0
3,44,45,0,1,0,0,0
4,55,20,0,0,0,0,1


2. Considere os dados do arquivo "german_credit.csv" que contem dados de empréstimos realizados por um banco.<br> <br>
a. Tem nulos nesse df? Se sim, preencha os nulos da forma que o grupo decidir.

In [70]:
df = pd.read_csv("data/german_credit.csv", index_col=0)
df.head()

,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose
0,67,male,2,own,NaN,little,1169,6,radio/TV
1,22,female,2,own,little,moderate,5951,48,radio/TV
2,49,male,1,own,little,NaN,2096,12,education
3,45,male,2,free,little,little,7882,42,furniture/equipment
4,53,male,2,free,little,little,4870,24,car


In [71]:
df.isna().sum()

Age                   0
Sex                   0
Job                   0
Housing               0
Saving accounts     183
Checking account    394
Credit amount         0
Duration              0
Purpose               0
dtype: int64

In [72]:
df["Saving accounts"].fillna(df["Saving accounts"].mode()[0], inplace=True)
df["Checking account"].fillna(df["Checking account"].mode()[0], inplace=True)
df.isna().sum()

Age                 0
Sex                 0
Job                 0
Housing             0
Saving accounts     0
Checking account    0
Credit amount       0
Duration            0
Purpose             0
dtype: int64

In [73]:
df.head()

,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose
0,67,male,2,own,little,little,1169,6,radio/TV
1,22,female,2,own,little,moderate,5951,48,radio/TV
2,49,male,1,own,little,little,2096,12,education
3,45,male,2,free,little,little,7882,42,furniture/equipment
4,53,male,2,free,little,little,4870,24,car


   b. Converta as variáveis categóricas em numéricas.

In [75]:
df = pd.get_dummies(df, columns=["Housing", "Saving accounts", "Checking account", "Purpose"])
df.replace({'Sex':{'male':'1', 'female':'0'}}, inplace=True)
df.head()

,Age,Sex,Job,Credit amount,Duration,Housing_free,Housing_own,Housing_rent,Saving accounts_little,Saving accounts_moderate,...,Checking account_moderate,Checking account_rich,Purpose_business,Purpose_car,Purpose_domestic appliances,Purpose_education,Purpose_furniture/equipment,Purpose_radio/TV,Purpose_repairs,Purpose_vacation/others
0,67,1,2,1169,6,0,1,0,1,0,...,0,0,0,0,0,0,0,1,0,0
1,22,0,2,5951,48,0,1,0,1,0,...,1,0,0,0,0,0,0,1,0,0
2,49,1,1,2096,12,0,1,0,1,0,...,0,0,0,0,0,1,0,0,0,0
3,45,1,2,7882,42,1,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0
4,53,1,2,4870,24,1,0,0,1,0,...,0,0,0,1,0,0,0,0,0,0


4. Para esse exercício vamos considerar o dataset https://archive.ics.uci.edu/ml/datasets/automobile que traz a especificação de modelos de carros em 1985.

In [69]:
# Considere a seguinte lista como o nome das colunas do dataset
headers = ["symboling", "normalized_losses", "make", "fuel_type", "aspiration",
           "num_doors", "body_style", "drive_wheels", "engine_location",
           "wheel_base", "length", "width", "height", "curb_weight",
           "engine_type", "num_cylinders", "engine_size", "fuel_system",
           "bore", "stroke", "compression_ratio", "horsepower", "peak_rpm",
           "city_mpg", "highway_mpg", "price"]
df=pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/autos/imports-85.data", header=None, names=headers)
df.head()

,symboling,normalized_losses,make,fuel_type,aspiration,num_doors,body_style,drive_wheels,engine_location,wheel_base,...,engine_size,fuel_system,bore,stroke,compression_ratio,horsepower,peak_rpm,city_mpg,highway_mpg,price
0,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495
1,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500
2,1,?,alfa-romero,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500
3,2,164,audi,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950
4,2,164,audi,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450


a. Esse dataset possui nans? Se sim, o grupo precisa entrar em um acordo de como tratá-los.

In [70]:
df.replace("?", np.nan, inplace = True)
df.isna().sum()

symboling             0
normalized_losses    41
make                  0
fuel_type             0
aspiration            0
num_doors             2
body_style            0
drive_wheels          0
engine_location       0
wheel_base            0
length                0
width                 0
height                0
curb_weight           0
engine_type           0
num_cylinders         0
engine_size           0
fuel_system           0
bore                  4
stroke                4
compression_ratio     0
horsepower            2
peak_rpm              2
city_mpg              0
highway_mpg           0
price                 4
dtype: int64

In [71]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 26 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   symboling          205 non-null    int64  
 1   normalized_losses  164 non-null    object 
 2   make               205 non-null    object 
 3   fuel_type          205 non-null    object 
 4   aspiration         205 non-null    object 
 5   num_doors          203 non-null    object 
 6   body_style         205 non-null    object 
 7   drive_wheels       205 non-null    object 
 8   engine_location    205 non-null    object 
 9   wheel_base         205 non-null    float64
 10  length             205 non-null    float64
 11  width              205 non-null    float64
 12  height             205 non-null    float64
 13  curb_weight        205 non-null    int64  
 14  engine_type        205 non-null    object 
 15  num_cylinders      205 non-null    object 
 16  engine_size        205 non

In [72]:
df["normalized_losses"] = pd.to_numeric(df["normalized_losses"])
df["horsepower"] = pd.to_numeric(df["horsepower"])
df["peak_rpm"] = pd.to_numeric(df["peak_rpm"])
df["price"] = pd.to_numeric(df["price"])
df["bore"] = pd.to_numeric(df["bore"])
df["stroke"] = pd.to_numeric(df["stroke"])

In [73]:
df["normalized_losses"].fillna(df["normalized_losses"].mean(), inplace=True)
df["num_doors"].fillna(df["num_doors"].mode()[0], inplace=True)
df["horsepower"].fillna(df["horsepower"].mean(), inplace=True)
df["peak_rpm"].fillna(df["peak_rpm"].mean(), inplace=True)
df["price"].fillna(df["price"].mean(), inplace=True)
df["bore"].fillna(df["bore"].mean(), inplace=True)
df["stroke"].fillna(df["stroke"].mean(), inplace=True)

In [74]:
df.isna().sum()

symboling            0
normalized_losses    0
make                 0
fuel_type            0
aspiration           0
num_doors            0
body_style           0
drive_wheels         0
engine_location      0
wheel_base           0
length               0
width                0
height               0
curb_weight          0
engine_type          0
num_cylinders        0
engine_size          0
fuel_system          0
bore                 0
stroke               0
compression_ratio    0
horsepower           0
peak_rpm             0
city_mpg             0
highway_mpg          0
price                0
dtype: int64

b. Esse dataset possui linhas duplicadas?

In [75]:
any(df.duplicated())

False

c. O consumo dos automóveis na cidade e rodovia, representados pelas colunas "city_mpg" e "highway_mpg" respectivamente, estão na unidade mpg (miles per gallon). Converta essas colunas para consumo em L/100km sabendo que a conversão é dada por:

L/100km = 235 / mpg 

In [76]:
from scipy import stats
print(stats.mode(np.array([1,1,1,2,2,2]), axis=None)[0])

[1]


In [38]:
df["city_lp100km"]=235/df["city_mpg"]
df["highway_lp100km"]=235/df["highway_mpg"]
df.head()

,symboling,normalized_losses,make,fuel_type,aspiration,num_doors,body_style,drive_wheels,engine_location,wheel_base,...,bore,stroke,compression_ratio,horsepower,peak_rpm,city_mpg,highway_mpg,price,city_lp100km,highway_lp100km
0,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,3.47,2.68,9.0,111,5000,21,27,13495,11.190476,8.703704
1,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,3.47,2.68,9.0,111,5000,21,27,16500,11.190476,8.703704
2,1,?,alfa-romero,gas,std,two,hatchback,rwd,front,94.5,...,2.68,3.47,9.0,154,5000,19,26,16500,12.368421,9.038462
3,2,164,audi,gas,std,four,sedan,fwd,front,99.8,...,3.19,3.40,10.0,102,5500,24,30,13950,9.791667,7.833333
4,2,164,audi,gas,std,four,sedan,4wd,front,99.4,...,3.19,3.40,8.0,115,5500,18,22,17450,13.055556,10.681818


d. Converta as colunas "body_style" e "drive_wheels" para variáveis dummies.

In [46]:
df.body_style.unique()

array(['convertible', 'hatchback', 'sedan', 'wagon', 'hardtop'],
      dtype=object)

In [49]:
df=pd.concat([df, pd.get_dummies(df["body_style"])], axis=1)
df.drop("body_style", axis = 1, inplace=True)
df.head()

,symboling,normalized_losses,make,fuel_type,aspiration,num_doors,drive_wheels,engine_location,wheel_base,length,...,horsepower,peak_rpm,city_mpg,highway_mpg,price,convertible,hardtop,hatchback,sedan,wagon
0,3,?,alfa-romero,gas,std,two,rwd,front,88.6,168.8,...,111,5000,21,27,13495,1,0,0,0,0
1,3,?,alfa-romero,gas,std,two,rwd,front,88.6,168.8,...,111,5000,21,27,16500,1,0,0,0,0
2,1,?,alfa-romero,gas,std,two,rwd,front,94.5,171.2,...,154,5000,19,26,16500,0,0,1,0,0
3,2,164,audi,gas,std,four,fwd,front,99.8,176.6,...,102,5500,24,30,13950,0,0,0,1,0
4,2,164,audi,gas,std,four,4wd,front,99.4,176.6,...,115,5500,18,22,17450,0,0,0,1,0


In [40]:
df.drive_wheels.unique()

array(['rwd', 'fwd', '4wd'], dtype=object)

In [50]:
df=pd.concat([df, pd.get_dummies(df["drive_wheels"])], axis=1)
df.drop("drive_wheels", axis = 1, inplace=True)
df.head()

,symboling,normalized_losses,make,fuel_type,aspiration,num_doors,engine_location,wheel_base,length,width,...,highway_mpg,price,convertible,hardtop,hatchback,sedan,wagon,4wd,fwd,rwd
0,3,?,alfa-romero,gas,std,two,front,88.6,168.8,64.1,...,27,13495,1,0,0,0,0,0,0,1
1,3,?,alfa-romero,gas,std,two,front,88.6,168.8,64.1,...,27,16500,1,0,0,0,0,0,0,1
2,1,?,alfa-romero,gas,std,two,front,94.5,171.2,65.5,...,26,16500,0,0,1,0,0,0,0,1
3,2,164,audi,gas,std,four,front,99.8,176.6,66.2,...,30,13950,0,0,0,1,0,0,1,0
4,2,164,audi,gas,std,four,front,99.4,176.6,66.4,...,22,17450,0,0,0,1,0,1,0,0


## Referências:
